In [1]:
import geopandas as gpd
import folium
import fiona
import json

In [4]:
# Enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['kml'] = 'rw' # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['KML'] = 'rw' # enable KML support which is disabled by default

# Load KML files using geopandas
path = './Visualization/'
kml_files = ['source.kml', 'true.kml', 'pred_LSTM.kml']  # Add paths to KML files
gdf_list = [gpd.read_file(path + kml) for kml in kml_files]

# Convert GeoDataFrames to GeoJSON strings
gdf_json_list = [gdf.to_crs(epsg='4326').to_json() for gdf in gdf_list]

# Parse the GeoJSON strings
gdf_geojson_list = [json.loads(gdf_json) for gdf_json in gdf_json_list]

# crea a list of colors to use for the layers
colors = ["blue", 'red', 'green']

# creat map frame
f = folium.Figure(width=500, height=500)

# Create a folium map with center location and zoom level on the map
map_obj = folium.Map(location=[gdf_geojson_list[0]['features'][0]['geometry']['coordinates'][0][1],
                               gdf_geojson_list[0]['features'][0]['geometry']['coordinates'][0][0]],
                     zoom_start=10).add_to(f)


# Add GeoJSON layers to the map
for i, gdf_geojson in enumerate(gdf_geojson_list):
    folium.GeoJson(
        gdf_geojson,
        name=kml_files[i].split('.')[0],
        style_function=lambda x, color=colors[i]: {
            'color': color,
            'weight': 2,
            'fillOpacity': 0.5,
        }
    ).add_to(map_obj)

# Add Layer Control to the map
folium.LayerControl().add_to(map_obj)

# Display the map
map_obj
